In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

tqdm().pandas()

#data=data.loc[data.userGamesPlayed > 50]

In [2]:
data=pd.read_csv('/w/225/1/chess/tactics/glicko_user_tactics_problem.csv_00')

In [3]:
puzzle_data = pd.read_csv('/w/225/1/chess/tactics/tactics_problem.csv')
data = pd.merge(data,puzzle_data[['tactics_problem_id','rating']],on='tactics_problem_id',how='inner')

In [7]:
data.rename(columns={'rating':'problemRating'},inplace=True)

In [8]:
data.head()

,user_tactics_problem_id,user_hash,create_date,date,last_game_date,seconds,is_passed,correct_move_count,rating_change,tactics_problem_id,ratingUser,varianceUser,ratingProblem,varianceProblem,userGamesPlayed,problemRating
0,38941,344494aff0032458cfe9096737c4cd13f829df9e7d3935...,2008-01-22 05:25:28,2008-01-22,2008-01-22,22,0,1,-12,810,1720,350,1720,350,0,687
1,381211,4017cdf33cd80adc79b3435a882fd8742075440b0ea0bf...,2008-01-30 02:34:19,2008-01-30,2008-01-30,30,0,1,-7,810,1429,265,1882,294,1,687
2,386511,72a1f7861e5d5fc47259095d561a2a8b3ddfad84f9dbad...,2008-01-30 06:47:40,2008-01-30,2008-01-30,42,1,3,4,810,1720,350,1921,273,0,687
3,414991,871d84076209702c0cda06da266b96d499d690638ee57e...,2008-01-30 17:07:19,2008-01-30,2008-01-30,48,0,0,-14,810,1720,350,1761,242,0,687
4,437161,f2e3d53ee5f731c2a9dffb61d33974216b8cf94eccf13b...,2008-01-31 05:22:34,2008-01-31,2008-01-24,16,1,3,7,810,1836,307,1847,220,1,687


In [9]:
trimmed_data = data[['seconds','is_passed','ratingUser','problemRating']]

In [19]:
ratings = [1400,1700,2000,2300]
rating_rad = 100
rating_ranges = []
for rating in ratings:
    rating_ranges.append((rating-rating_rad,rating+rating_rad))
print(rating_ranges)

[(1300, 1500), (1600, 1800), (1900, 2100), (2200, 2400)]


In [46]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

def plot_graph(data,puzz_rating):
    figure(figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
    colors = ['#1dc23b','#f28d18','#931dc2','#c21d1d']
    group = ['1400','1700','2000','2300']

    for index,data in enumerate(probs):
        color = colors[index]
        plt.plot(data,color=color,label=group[index])
        
    plt.ylabel('Probability of success')
    plt.xlabel('Time waited >= x')
    plt.title('Player Success Over time at Puzzles around {}'.format(puzz_rating))
    plt.legend(loc=2)
    plt.savefig('../plots/player_time_success_{}_puzzles.png'.format(puzz_rating))
    plt.clf()

In [47]:
time_range = list(range(0,40))
# if A = # of correct attempts and B = # of problems where seconds >= x time and ratings for user and problem are 
#within a certain range. Want to find: P(A|B) = P(A&B)/P(B)
for puz_rating in tqdm(rating_ranges):
    probs = []
    print('Generating plot for puzzles around: ' + str(puz_rating[0]+rating_rad))
    for tup in rating_ranges:
        print('Doing ratings for players around: ' + str(tup[0]+rating_rad))
        rating_probs = []
        #Get problems within the correct rating range
        rating_trimmed = trimmed_data.loc[(trimmed_data.problemRating.between(puz_rating[0],puz_rating[1])) & 
                                             (trimmed_data.ratingUser.between(tup[0],tup[1]))]
        #Filter the problemd from rating_trimmed that were attempted after at least time seconds
        for time in time_range:
            b_condition = rating_trimmed.loc[rating_trimmed.seconds >= time]
            rating_probs.append(len(b_condition.loc[b_condition.is_passed == 1])/len(b_condition))
        probs.append(rating_probs)
    #Making plot
    plot_graph(probs,puz_rating[0]+rating_rad)

Generating plot for puzzles around: 1400
Doing ratings for players around: 1400
Doing ratings for players around: 1700
Doing ratings for players around: 2000
Doing ratings for players around: 2300
Generating plot for puzzles around: 1700
Doing ratings for players around: 1400
Doing ratings for players around: 1700
Doing ratings for players around: 2000
Doing ratings for players around: 2300
Generating plot for puzzles around: 2000
Doing ratings for players around: 1400
Doing ratings for players around: 1700
Doing ratings for players around: 2000
Doing ratings for players around: 2300
Generating plot for puzzles around: 2300
Doing ratings for players around: 1400
Doing ratings for players around: 1700
Doing ratings for players around: 2000
Doing ratings for players around: 2300


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>